Import package
CCTV的前處理
(!pip install opencv-python)

In [5]:
import numpy as np
import cv2
from datetime import datetime, timedelta
import os, threading

Define read2frame
目的 : 將圖片xy的0.3 並轉成灰階，同時計算frame的數量
return值為一四維的np.array，(數量,X,Y,channel)
因為每讀到一楨frame，f就append，也因此function的return值是四維
同時len(f)也就是其frame的數量
另外，因為轉成灰階，其channel為1，原本灰階圖片的np.array是(x,y)，而非灰階維(x,y,channel=3)
因為要丟進model train，其convolution的參數要為4維(看document)
也因此，多加一維變成(x,y,channel=1)

In [6]:
def read2frame(v):
    resize_rate = 0.3
    f = []
    t = []
    cap = cv2.VideoCapture(v)
    ret, frame = cap.read()
    count = 1
    while ret:
        try:
            ret, frame = cap.read()
            if not ret:
                break
            # count += 1
            # if count % 2 == 0:
            #     continue
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            # frame = cv2.equalizeHist(frame)
            # ret, frame = cv2.threshold(frame, 127, 255, cv2.THRESH_BINARY)
            # t.append(frame[190:203, 322:335])
            # ff = (ff - ff.mean()) / ff.std()
            frame = cv2.resize(frame, (0, 0), fx=resize_rate, fy=resize_rate)
            # if len(f) < frame_num:
                # if c == 'nfbCCTV-N5-S-21.048-M':
                #     frame = frame[int(np.shape(frame)[0] * (2 / 11)):, int(np.shape(frame)[1] * (2 / 7)):]
                # elif c == 'nfbCCTV-N5-S-23.896-M':
                #     frame = frame[int(np.shape(frame)[1] * (1 / 13)):, int(np.shape(frame)[1] * (2 / 11)):]
                # else:
                #     frame = frame[int(np.shape(frame)[0] * (2 / 7)):, int(np.shape(frame)[1] * (2 / 13)):]
            f.append(frame)

            # else:
            #     break
        except:
            pass
    # back = np.zeros_like(f[0])
    # back2 = np.zeros_like(t[0])
    # print(count)
    # if count < 180:
    #     return None
    # while len(f) < frame_num:
    #     f.append(back)
    #     t.append(back2)
    # print(np.shape(f))
    cap.release()
    cv2.destroyAllWindows() #非灰階圖片有三個channel (x,y,3) 威接圖片維兩個維度(x,y)
    return np.expand_dims(f, axis=4) #===============必須要轉成4個維度，因為轉成灰階後會少了ｃｈａｎｎｅｌ
#===============================================所以4個維度分別是(數量、frame的 x、y、以及1(channel的數量))


cctvs : 要前處理的CCTVs，可自行調整

In [ ]:
cctvs = ['nfbCCTV-N5-N-16.915-M', 'nfbCCTV-N5-N-18.308-M', 'nfbCCTV-N5-N-19.707-M', 'nfbCCTV-N5-N-21.056-M',
         'nfbCCTV-N5-N-22.514-M', 'nfbCCTV-N5-N-23.921-M', 'nfbCCTV-N5-N-25.309-M',
         'nfbCCTV-N5-S-16.892-M', 'nfbCCTV-N5-S-18.339-M', 'nfbCCTV-N5-S-19.7-M', 'nfbCCTV-N5-S-21.048-M',
         'nfbCCTV-N5-S-22.493-M', 'nfbCCTV-N5-S-23.896-M', 'nfbCCTV-N5-S-25.298-M']
cctvs = ['nfbCCTV-N5-N-16.915-M', 'nfbCCTV-N5-N-18.308-M', 'nfbCCTV-N5-N-22.514-M', 'nfbCCTV-N5-N-25.309-M',
         'nfbCCTV-N5-S-18.339-M', 'nfbCCTV-N5-S-19.7-M', 'nfbCCTV-N5-S-21.048-M', 'nfbCCTV-N5-S-23.896-M']
cctvs = ['nfbCCTV-N5-S-21.048-M', 'nfbCCTV-N5-S-23.896-M', 'nfbCCTV-N5-N-25.309-M']
# cctvs = ['nfbCCTV-N5-S-23.896-M']


day1 : 起始日期
dayLast : 結束日期
dont_care_start ,dont_care_end : 不需要的時間區段 (目前維0600-2400)

In [ ]:
day1 = datetime.strptime('20190729 0600', "%Y%m%d %H%M")
dayLast = datetime.strptime('20190730 2359', "%Y%m%d %H%M")

dont_care_start = datetime.strptime('0000', "%H%M")
dont_care_end = datetime.strptime('0600', "%H%M")


E:/wcs/cctv/的檔案路徑改為D:/wcs/cctv/(我設定的 學長預設E槽)

In [ ]:
# frame_num = 90

for cctv_Id in cctvs:
    print(cctv_Id)
    cctvDir = 'D:/cctv/' + cctv_Id + "/"
    os.chdir(cctvDir)
    if not os.path.exists("D:/wcs/cctv/" + cctv_Id):
        os.makedirs("D:/wcs/cctv/" + cctv_Id)
    day = day1
    file_name = "D:/wcs/cctv/" + cctv_Id + "/frame_num.txt"
    open(file_name, "w").close()
    while day <= dayLast:
        if not os.path.exists(day.strftime("%Y%m%d") + "_" + day.strftime("%H%M") + "20") \
                or dont_care_start.hour <= day.hour < dont_care_end.hour:
            day += timedelta(minutes=1)
            continue
        frames = read2frame(day.strftime("%Y%m%d") + "_" + day.strftime("%H%M") + "20")
        print(day)
        print(day.strftime("%Y/%m/%d %H:%M:%S") + "," + repr(len(frames)), file=open(file_name, 'a'))
        if frames is not None and len(frames) > 1:
            np.save('D:/wcs/cctv/' + cctv_Id + "/" + day.strftime("%Y%m%d") + "_" + day.strftime("%H%M"), frames)
            # np.save('E:/wcs/cctv/' + cctv_Id + "/" + day.strftime("%Y%m%d") + "_" + day.strftime("%H%M") + "_timestamp", times)
        day += timedelta(minutes=1)
